# Le notebook que yohan n'a pas fait

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [4]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]
os.mkdir('./Dataset/all_years/', 0o666) 
for file_name in files_name:
    result = []
    print(f"\n///////////////{file_name}///////////////")
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
        print(df.columns.tolist())
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)


///////////////caracteristiques-20///////////////
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']


OSError: Cannot save file into a non-existent directory: 'Dataset\all_years'

In [3]:
data = pd.read_csv('accidents-19-22.csv', low_memory=False)

pd.DataFrame(data)

FileNotFoundError: [Errno 2] No such file or directory: 'accidents-19-22.csv'

In [ ]:
df = pd.DataFrame(data.groupby(['Accident_Id']))

df[0]

0         (201900000001,)
1         (201900000002,)
2         (201900000003,)
3         (201900000004,)
4         (201900000005,)
               ...       
218399    (202200055298,)
218400    (202200055299,)
218401    (202200055300,)
218402    (202200055301,)
218403    (202200055302,)
Name: 0, Length: 218404, dtype: object